# This notebook is to test a single batch run in ADAM

In [1]:
from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
import time
import os

This sets up authenticated access to the server. It needs to be done before pretty much everything you want to do with ADAM.

In [2]:
# ConfigManager loads the config set up via adamctl.
# See the README at https://github.com/B612-Asteroid-Institute/adam_home/blob/master/README.md
config = ConfigManager().get_config()
auth_rest = AuthenticatingRestProxy(RestRequests(config['url']), config['token'])

## Example Inputs

In [3]:
# 6x1 state vector (position [km], velocity [km/s])
state_vec = [130347560.13690618,
             -74407287.6018632,
             -35247598.541470632,
             23.935241263310683,
             27.146279819258538,
             10.346605942591514]

# Lower triangular covariance matrix (21 elements in a list)
covariance = [3.331349476038534e-04, + \
              4.618927349220216e-04, 6.782421679971363e-04, + \
             -3.070007847730449e-04, -4.221234189514228e-04, 3.231931992380369e-04, + \
             -3.349365033922630e-07, -4.686084221046758e-07, 2.484949578400095e-07, 4.296022805587290e-10, + \
             -2.211832501084875e-07, -2.864186892102733e-07, 1.798098699846038e-07, 2.608899201686016e-10, 1.767514756338532e-10, + \
             -3.041346050686871e-07, -4.989496988610662e-07, 3.540310904497689e-07, 1.869263192954590e-10, 1.008862586240695e-10, 6.224444338635500e-10]

### Set Parameters

Commented parameters are optional. Uncomment to use.

In [4]:
propagation_params = PropagationParams({
    'start_time': '2017-10-04T00:00:00Z',   # propagation start time in ISO format
    'end_time': '2017-10-11T00:00:00Z',     # propagation end time in ISO format

    'project_uuid': config['workspace'],

#     'step_size': 60 * 60,                   # step size (seconds)
#     'propagator_uuid': '00000000-0000-0000-0000-000000000002',  # force model
#     'description': 'some description'       # description of run
})

opm_params = OpmParams({
    'epoch': '2017-10-04T00:00:00Z',
    'state_vector': state_vec,
            
#     'mass': 500.5,              # object mass
#     'solar_rad_area': 25.2,     # object solar radiation area (m^2)
#     'solar_rad_coeff': 1.2,     # object solar radiation coefficient
#     'drag_area': 33.3,          # object drag area (m^2)
#     'drag_coeff': 2.5,          # object drag coefficient
    
#     'covariance': covariance,   # object covariance
#     'perturbation': 3,          # sigma perturbation on state vector
#     'hypercube': 'FACES',       # hypercube propagation type
    
#     'originator': 'Robot',      # originator of run
#     'object_name': 'TestObj',   # object name
#     'object_id': 'test1234',    # object ID
})

### Submit and Run Propagation

In [5]:
batch = Batch(propagation_params, opm_params)
print("Submitting OPM:")
print(batch.get_opm_params().generate_opm())

Submitting OPM:
CCSDS_OPM_VERS = 2.0
CREATION_DATE = 2020-01-31 01:42:45.015138
ORIGINATOR = ADAM_User
COMMENT Cartesian coordinate system
OBJECT_NAME = dummy
OBJECT_ID = 001
CENTER_NAME = SUN
REF_FRAME = ICRF
TIME_SYSTEM = UTC
EPOCH = 2017-10-04T00:00:00Z
X = 130347560.13690618
Y = -74407287.6018632
Z = -35247598.54147063
X_DOT = 23.935241263310683
Y_DOT = 27.146279819258538
Z_DOT = 10.346605942591514
MASS = 1000.0
SOLAR_RAD_AREA = 20.0
SOLAR_RAD_COEFF = 1.0
DRAG_AREA = 20.0
DRAG_COEFF = 2.2



In [6]:
# Submit and wait until batch run is ready
batches_module = Batches(auth_rest)
BatchRunManager(batches_module, [batch]).run()

[17.709584] Submitting 1 runs.
[6.314619] Running.
[0.517325] Retrieving propagation results.


### Get Status and Parts Count

In [7]:
# Get final status and parts count
parts_count = batch.get_state_summary().get_parts_count()
print("Final state: %s, part count %s\n" % (batch.get_calc_state(), parts_count))

Final state: COMPLETED, part count 1



### Get Ephemeris of Specified Part

In [8]:
# Get ephemeris of specified part
part_to_get = 0
eph = batch.get_results().get_parts()[part_to_get].get_ephemeris()
print("Ephemeris:")
print(eph)

Ephemeris:
stk.v.9.0

# WrittenBy    STK_Components_2018 r3(18.3.395.0)

BEGIN Ephemeris

NumberOfEphemerisPoints	8
ScenarioEpoch	4 Oct 2017 00:00:00
InterpolationMethod	Hermite
InterpolationSamplesM1	2
CentralBody	Sun
CoordinateSystem	ICRF

EphemerisTimePosVel

0 130347560136.90617 -74407287601.8632 -35247598541.470634 23935.241263310683 27146.27981925854 10346.605942591514
86400 132397901142.35962 -72051923842.2795 -34348940383.26344 23525.854645014435 27374.492096848546 10455.050276306205
172800 134412746667.10405 -69677241421.15378 -33441072650.973186 23113.65011574464 27593.46699066004 10559.786619242846
259200 136391869646.87138 -67284039250.353264 -32524316622.08983 22699.006699408754 27803.188844064465 10660.795195643448
345600 138335075572.65045 -64873116848.67935 -31598994968.59646 22282.29878343753 28003.659175729463 10758.06347125273
432000 140242202043.74875 -62445272881.82381 -30665431139.52489 21863.895057395603 28194.89611561114 10851.585958183761
518400 142113118232.18

### Get ending state vector

In [9]:
# Get the end state vector (uncomment to use)
end_state_vector = batch.get_results().get_end_state_vector()
print("State vector at the end of propagation:")
print(end_state_vector)

State vector at the end of propagation:
[143947724.264667, -57542002.254282884, -28774871.060081646, 21.023440575257016, 28.549821594856, 11.027405485916118]
